# Import Libraries

In [52]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords

# Read 100 questions list

In [53]:
df = pd.read_excel('AnswersFromSource11_20.xlsx',engine='openpyxl')
link = 'https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx'
df[link]=np.nan
#df.drop(df.iloc[:, 1:25], inplace = True, axis = 1)
df

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN
...,...,...,...
104,What are some ways that parents can reduce the...,NaN,NaN
105,Do some families deal with stress better than ...,NaN,NaN
106,Do siblings suffer increased stress as a resul...,NaN,NaN
107,What can I do about my children’s stress?,NaN,NaN


# Scrape QA pairs from website

In [54]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx')



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/home/gopan/.wdm/drivers/chromedriver/linux64/92.0.4515.107/chromedriver] found in cache


In [55]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [56]:
# Search the questions mentioned in webpage
l = []
for i in soup.find_all('div',{'class':'rc_header'}):
    #if(i.text[-1]=='?'):
    print(i.text)
    l.append(i.text)

What is autism? 
What are early signs of autism? 
What are some symptoms of autism that parents and caregivers can look for ?
What is the difference between autism and Asperger's disorder?
Are there treatments available for autism?
Are there medication treatments for autism?
Why do children with autism have difficulty learning in a regular classroom setting?
What is the difference between autism and pervasive developmental disorder, not otherwise specified? 


In [57]:
# Tried to search for answers of questions
k=[]
c = 0
for i in soup.find_all('p'):
    txt = i.text.strip()
    if(txt in l):
        print('*'*100)
    if(txt in l or len(txt.split())>10):
        print(c,'-->', txt)
    c = c+1
    
    k.append(txt)

0 --> Autism is a neurodevelopmental condition which is usually diagnosed in the first 3 years of life. Generally parents become concerned when their child has delays in speech development, limited social relatedness, and restricted interests and activities. The child may avoid direct eye contact and exhibit odd behaviors such as focusing on parts of objects (e.g. the spinning wheel of a toy car). There may be unusual motor movements such as hand flapping, self stimulation or walking on toes.
1 --> Although the cause of autism is unknown, it is generally believed that etiology may be due to multiple factors. Many genetic, environmental, metabolic and neurological conditions that affect the normal functioning of the brain are being researched. The diagnosis of autism requires disturbances in each of three domains: (1) social relatedness, (2) communication/play, and (3) restricted interests and activities.
2 --> Treatment planning is complex since each child has different strengths and d

In [58]:
lis = []
for ul in soup.find_all('ul'):
    for li in ul.findAll('li'):
        if li.find('ul'):
            break
        lis.append(li.text)
lis = lis[44:]
for li in lis:
    print(li)
print(lis[28])    

Social relatedness includes marked impairment in non-verbal communication, peer relationships and social-emotional reciprocity. 
Communication/play includes either a delay or total lack of spoken language and lack of developmentally-appropriate make-believe or social play. 
Restricted interests and activities includes encompassing preoccupations, adherence to non-functional routines or rituals, stereotypies and motor mannerisms. 
lack of or delay in development of spoken language .
repetitive use of language and/or motor mannerisms (e.g., hand-flapping, twirling objects). 
little or no eye contact. 
lack of interest in peer relationships. 
lack of spontaneous or make-believe play. 
persistent fixation on parts of objects.
Insistence on sameness in routines (O) 
Difficulty in expressing needs verbally, using gestures or pointing instead of words (C) 
Repeating words or phrases in place of normal, responsive language (C)
Laughing (and/or crying) for no apparent reason; showing distress f

# Check which questions are similar

In [59]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('Autism Spectrum Disorder','ASD',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [60]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [61]:
for i in l:
    i2 = clean_text(i)
    list1 = i2.split()
    
    temp1 = 0.0
    temp2 = ''
    temp3 = ''
    count = 0
    for z1, j in enumerate(df['Question']):
        j2 = clean_text(j)
        list2 = j2.split()
            
        sim = jaccard_similarity(list1, list2)
        
        if(sim>temp1):
            temp1 = sim
            temp2 = j
            temp3 = i
            count = z1+1
    if(temp1>=0.3):
        print('Website --> ',temp3)
        print('100 questions[ ',count,' ] list --> ',temp2)
        print(temp1)
        print('*'*100)

Website -->  What is autism? 
100 questions[  10  ] list -->  What is Autism?
1.0
****************************************************************************************************
Website -->  What are early signs of autism? 
100 questions[  10  ] list -->  What is Autism?
0.3333333333333333
****************************************************************************************************
Website -->  What are some symptoms of autism that parents and caregivers can look for ?
100 questions[  23  ] list -->  What are some of the symptoms of Autism a Parent should look for?
0.5
****************************************************************************************************
Website -->  What is the difference between autism and Asperger's disorder?
100 questions[  13  ] list -->  How can you tell Autism from Asperger’s Syndrome?
0.3333333333333333
****************************************************************************************************
Website -->  Are there treatments

In [62]:
# Directly add answer to questions in existing dataframe
#df.iloc[9,0]
#What is Autism?
df[link].loc[9]= k[0] + "\n" + k[1]+"\n 1. "+lis[0]+"\n 2. "+lis[1]+"\n 3. "+lis[2] +"\n"+ k[2]
#print(df.iloc[9,1])

#What are some of the symptoms of Autism a Parent should look for?
df[link].loc[23] = k[4]+"\n 1. "+lis[9]+"\n 2. "+lis[10]+"\n 3. "+lis[11] +"\n 4. "+ lis[12]+"\n 5. "+lis[13]+"\n 6. "+lis[14] +"\n 7. "+ lis[15]+"\n 8. "+lis[16]+"\n 9. "+lis[17]+"\n 10. "+lis[18] +"\n 11. "+ lis[19]+"\n 12. "+lis[20]+"\n 13. "+lis[21] +"\n 14. "+ lis[22]+"\n 15. "+lis[23]+"\n 16. "+lis[24]+"\n 17. "+lis[25] +"\n 18. "+ lis[26]+"\n "+k[5]
#print(df.iloc[23,1])

#

/home/gopan/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [63]:
l

['What is autism? ',
 'What are early signs of autism? ',
 'What are some symptoms of autism that parents and caregivers can look for ?',
 "What is the difference between autism and Asperger's disorder?",
 'Are there treatments available for autism?',
 'Are there medication treatments for autism?',
 'Why do children with autism have difficulty learning in a regular classroom setting?',
 'What is the difference between autism and pervasive developmental disorder, not otherwise specified? ']

In [64]:
len(l)
l1 = [l[1],l[3],l[4],l[5],l[6]]
l1

['What are early signs of autism? ',
 "What is the difference between autism and Asperger's disorder?",
 'Are there treatments available for autism?',
 'Are there medication treatments for autism?',
 'Why do children with autism have difficulty learning in a regular classroom setting?']

In [65]:
# manually select answers to questions in list 'l1'

l2 = []
l2.append(k[3] + "\n 1. " +lis[3]+"\n 2. "+lis[4]+"\n 3. "+lis[5] +"\n 4. "+ lis[6]+"\n 5. "+lis[7]+"\n 6. "+lis[8])
l2.append(k[6] + "\n" + k[7]+ "\n" + k[8]+ "\n" + k[9])
l2.append(k[10])
l2.append(k[11])
l2.append(k[12] +  "\n 1. " +lis[27]+"\n 2. "+lis[28]+"\n 3. "+lis[29] +"\n 4. "+ lis[30]+"\n 5. "+lis[31]+"\n 6. "+lis[32]+"\n 7. "+lis[33]+ "\n "+ k[13])

In [66]:
# Create new dataframe with QA pairs
df2 = pd.DataFrame(zip(l1,l2),columns=['Question',link])
df2

,Question,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx
0,What are early signs of autism?,Parents and relatives should be concerned abou...
1,What is the difference between autism and Aspe...,"Unlike children with autism, individuals with ..."
2,Are there treatments available for autism?,There are no specific treatments to “cure” aut...
3,Are there medication treatments for autism?,There are no medication treatments that treat ...
4,Why do children with autism have difficulty le...,There are many reasons that a child diagnosed ...


In [67]:
# Concatenate existing and new dataframes
df3 = pd.concat([df,df2],axis=0)
df3 =df3.sort_values(by=list(df3.columns[1:])).reset_index(drop=True)
df3

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN
...,...,...,...
109,What are some ways that parents can reduce the...,NaN,NaN
110,Do some families deal with stress better than ...,NaN,NaN
111,Do siblings suffer increased stress as a resul...,NaN,NaN
112,What can I do about my children’s stress?,NaN,NaN


# Save Output

In [68]:
df3.to_excel('AnswersFromSource11_20.xlsx',index=False)